In [25]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [26]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


In [27]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

In [28]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Save Checkpoints During Training

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) # pass callback to training



Epoch 1/10
32/32 [==============================] - 1s 11ms/step - loss: 1.2223 - sparse_categorical_accuracy: 0.6330 - val_loss: 0.7624 - val_sparse_categorical_accuracy: 0.7690

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.4431 - sparse_categorical_accuracy: 0.8700 - val_loss: 0.5611 - val_sparse_categorical_accuracy: 0.8280

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.3073 - sparse_categorical_accuracy: 0.9170 - val_loss: 0.4773 - val_sparse_categorical_accuracy: 0.8610

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2074 - sparse_categorical_accuracy: 0.9560 - val_loss: 0.4406 - val_sparse_categorical_accuracy: 0.8480

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 0.1682 - sparse_cat

In [30]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [31]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3174 - sparse_categorical_accuracy: 0.1040
Untrained model, accuracy: 10.40%


In [32]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3979 - sparse_categorical_accuracy: 0.8690
Restored model, accuracy: 86.90%


In [33]:
# Checkpoint callback options

# Include the epoch in the file name(uses 'str.format')
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# save new checkpoint every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the 'checkpoint_path' format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size = batch_size,
          callbacks=[cp_callback],
          validation_data = (test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [34]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [35]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4958 - sparse_categorical_accuracy: 0.8740
Restored model, accuracy: 87.40%


In [44]:
# Manually Saving Weights (default stored as .ckpt file)
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4141 - sparse_categorical_accuracy: 0.8650
Restored model, accuracy: 86.50%


In [45]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1257 - sparse_categorical_accuracy: 0.6740
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4482 - sparse_categorical_accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2768 - sparse_categorical_accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2003 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1483 - sparse_categorical_accuracy: 0.9710
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [50]:
# my_model directory
ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
ls saved_model/my_model

SyntaxError: ignored

In [47]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Predict with loaded model/ Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4176 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.30%
(1000, 10)


In [51]:
# HDF5 Saving Format

# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

# Save the entire model to a HDF5 file
# The '.h5' extension indicates the model should be saved to HDF5
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1441 - sparse_categorical_accuracy: 0.6860
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4149 - sparse_categorical_accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2817 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1987 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1429 - sparse_categorical_accuracy: 0.9760


In [53]:
# Recreate the exact same model, including its weights and optimizer from saved file
new_model  = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Check its accuracy
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4498 - sparse_categorical_accuracy: 0.8440
Restored model, accuracy: 84.40%
